In [ ]:
# default_exp datatypes.circuit

# Circuit

> Representation of a quantum circuit.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
from collections.abc import MutableSequence
from functools import lru_cache
import hashlib

In [ ]:
#export
GATESETS = {
    'gate1': {'H', 'X', 'Y', 'Z', 'I', 'S', 'Sd', 'T', 'Td', 'Q', 'Qd', 'R', 'Rd'},
    'gate2': {'CNOT', 'MSd'},
    'meas': {'measure'},
    'init': {'init'}
}

A circuit is a list of ticks. A tick consists of gates associated to qubits. We can have one-qubit gates which are applied to single gate locations and two-qubit gates which are applied to an ordered (control,target)-qubit pair. For simplicity, we implement this as a list of dictionaries. Each tick is represented by a dictionary (a *tick*tionary if you may) which holds the `gate_symbol` as key and the set of `qubits` we want to apply these gates to as values. Qubits can be either *integers* for single-qubit locations or *tuples* for two-qubit locations.

In [ ]:
#export
class Circuit(MutableSequence):
    """Representation of a quantum circuit"""
    
    def __init__(self, ticks=None, noisy=True, ff_det=False):
        self._ticks = ticks if ticks else []
        self._noisy = noisy
        self._ff_deterministic = ff_det # fault-free deterministic
        
    def __getitem__(self, tick_index):
        return self._ticks[tick_index]
    
    def __setitem__(self, tick_index, tick):
        self._ticks[tick_index] = tick
        
    def __delitem__(self, tick_index):
        del self._ticks[tick_index]
        
    def __len__(self):
        return len(self._ticks)
    
    def insert(self, tick_index, tick):
        self._ticks.insert(tick_index, tick)
    
    def __str__(self):
        str_list = []
        for i, tick in enumerate(self._ticks):
            str_list.append(f"{i}: {str(tick)}")
        return "\n".join(str_list)
    
    def __repr__(self):
        return self.__str__()
    
    @property
    def _qubits(self):  
        """Set of qubits used in circuit"""
        return set(unpack(self._ticks))
    
    @property
    @lru_cache(maxsize=1)
    def n_qubits(self):
        """Number of qubits used in circuit"""
        return len(self._qubits)
    
    @property
    def n_ticks(self):
        """Number of ticks"""
        return len(self._ticks)
    
    @property
    def partition(self):
        """Partition circuit into gatesets+idle groups"""
        qubits = set(range(self.n_qubits))
        out = {'idle': []}
        get_partition = lambda gset: [(i,q) for i,t in enumerate(self)
                                       for g,qs in t.items() for q in qs if g in gset]
        
        for gname,gset in GATESETS.items():
            out[gname] = get_partition(gset)
            
        for i,tick in enumerate(self):
            out['idle'] += [(i,q) for q in qubits.difference(set(unpack(tick)))]
            
        return out

In [ ]:
circuit = Circuit([{'Y': [0,3]}, {'X': [0,1], 'CNOT': [(2,3)]}, {'CNOT': [(2,3),(0,1)]}])
par = circuit.partition

# partition = [subset_gate1_locs, subset_gate2_locs, subset_meas_locs, ..]

import itertools as it
import qsam.math as math

# ...
def protocol_subset_occurence(partitions: dict, partition_weights: dict, p_phys: (list,np.ndarray,float)) -> dict:
    """Generate subset occurence probabilities corresponding to each subset weight
    combination per circuit partition in partitions"""
    
    def circuit_subset_occurence(partition, partition_weight, p_phy_per_subset):
        subset_lens = np.array([len(locs) for locs in partition.values()])
        Aws = np.array([math.binom(w_vec, subset_lens, p_phy_per_subset) for w_vec in partition_weight])
        Aws = np.product(Aws, axis=-1)
        return Aws

    return {chash: circuit_subset_occurence(partitions[chash],partition_weight,p_phys)
            for chash, partition_weight in partition_weights.items()}

def protocol_partition_weights(partitions: dict) -> dict:
    """Generate all possible subset weights per circuit partition in partitions"""
    
    def partition_weight_combs(subset_lens):
        return list(it.product( *[tuple(range(l+1)) for l in subset_lens] ))
    
    return {chash: partition_weight_combs([len(subset) for subset in partition.values()])
            for chash, partition in partitions.items()}

print(par)
p_weights = protocol_partition_weights({'s1': par})
print(p_weights)
protocol_subset_occurence({'s1': par}, p_weights, np.array([0.1]))

{'idle': [(0, 1), (0, 2)], 'gate1': [(0, 0), (0, 3), (1, 0), (1, 1)], 'gate2': [(1, (2, 3)), (2, (2, 3)), (2, (0, 1))], 'meas': [], 'init': []}
{'s1': [(0, 0, 0, 0, 0), (0, 0, 1, 0, 0), (0, 0, 2, 0, 0), (0, 0, 3, 0, 0), (0, 1, 0, 0, 0), (0, 1, 1, 0, 0), (0, 1, 2, 0, 0), (0, 1, 3, 0, 0), (0, 2, 0, 0, 0), (0, 2, 1, 0, 0), (0, 2, 2, 0, 0), (0, 2, 3, 0, 0), (0, 3, 0, 0, 0), (0, 3, 1, 0, 0), (0, 3, 2, 0, 0), (0, 3, 3, 0, 0), (0, 4, 0, 0, 0), (0, 4, 1, 0, 0), (0, 4, 2, 0, 0), (0, 4, 3, 0, 0), (1, 0, 0, 0, 0), (1, 0, 1, 0, 0), (1, 0, 2, 0, 0), (1, 0, 3, 0, 0), (1, 1, 0, 0, 0), (1, 1, 1, 0, 0), (1, 1, 2, 0, 0), (1, 1, 3, 0, 0), (1, 2, 0, 0, 0), (1, 2, 1, 0, 0), (1, 2, 2, 0, 0), (1, 2, 3, 0, 0), (1, 3, 0, 0, 0), (1, 3, 1, 0, 0), (1, 3, 2, 0, 0), (1, 3, 3, 0, 0), (1, 4, 0, 0, 0), (1, 4, 1, 0, 0), (1, 4, 2, 0, 0), (1, 4, 3, 0, 0), (2, 0, 0, 0, 0), (2, 0, 1, 0, 0), (2, 0, 2, 0, 0), (2, 0, 3, 0, 0), (2, 1, 0, 0, 0), (2, 1, 1, 0, 0), (2, 1, 2, 0, 0), (2, 1, 3, 0, 0), (2, 2, 0, 0, 0), (2, 2, 1, 0, 0)

{'s1': array([3.87420489e-01, 1.29140163e-01, 1.43489070e-02, 5.31441000e-04,
        1.72186884e-01, 5.73956280e-02, 6.37729200e-03, 2.36196000e-04,
        2.86978140e-02, 9.56593800e-03, 1.06288200e-03, 3.93660000e-05,
        2.12576400e-03, 7.08588000e-04, 7.87320000e-05, 2.91600000e-06,
        5.90490000e-05, 1.96830000e-05, 2.18700000e-06, 8.10000000e-08,
        8.60934420e-02, 2.86978140e-02, 3.18864600e-03, 1.18098000e-04,
        3.82637520e-02, 1.27545840e-02, 1.41717600e-03, 5.24880000e-05,
        6.37729200e-03, 2.12576400e-03, 2.36196000e-04, 8.74800000e-06,
        4.72392000e-04, 1.57464000e-04, 1.74960000e-05, 6.48000000e-07,
        1.31220000e-05, 4.37400000e-06, 4.86000000e-07, 1.80000000e-08,
        4.78296900e-03, 1.59432300e-03, 1.77147000e-04, 6.56100000e-06,
        2.12576400e-03, 7.08588000e-04, 7.87320000e-05, 2.91600000e-06,
        3.54294000e-04, 1.18098000e-04, 1.31220000e-05, 4.86000000e-07,
        2.62440000e-05, 8.74800000e-06, 9.72000000e-07, 3.

As `Circuit` is a `MutableSequence` we get all the Pythonic list functionality for free. Let's check several ways we can create and add to a circuit.<br>

**Note:** It is important to define the values as a **list** of qubits, and not as a **set**. Although both work, for several measurements in one tick only lists maintain the order in which these measurements take place, whereas for sets the qubit order is changed to increasing value. For example {2,1,3} will become {1,2,3}. Thus the output string of the `Simulator.run()` method will also follow this order when defining qubits in circuits as sets.

In [ ]:
circuit = Circuit([{'H': [1,2,3]}])
assert circuit._ticks == [{'H': [1,2,3]}]

In [ ]:
circuit.append({'X': [0,1], 'CNOT': [(2,3)]})
assert circuit._ticks == [{'H': [1,2,3]}, {'X': [0,1], 'CNOT': [(2,3)]}]

In [ ]:
circuit += [{'CNOT': [(2,3),(0,1)]}]
assert circuit._ticks == [{'H': [1,2,3]}, {'X': [0,1], 'CNOT': [(2,3)]}, {'CNOT': [(2,3),(0,1)]}]

In [ ]:
circuit[0] = {'Y': [0,3]}
assert circuit._ticks == [{'Y': [0,3]}, {'X': [0,1], 'CNOT': [(2,3)]}, {'CNOT': [(2,3),(0,1)]}]

Of course we can also print our `Circuit`. We get a list of ticks and the corresponding ticks which are sequentially applied to our state during simulation.

In [ ]:
print(circuit)

0: {'Y': [0, 3]}
1: {'X': [0, 1], 'CNOT': [(2, 3)]}
2: {'CNOT': [(2, 3), (0, 1)]}


Furthermore, our `Circuit` class gives us some useful information about the circuit, as

In [ ]:
show_doc(Circuit.n_qubits)
show_doc(Circuit.n_ticks)

<h4 id="Circuit.n_qubits" class="doc_header"><code>Circuit.n_qubits</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Number of qubits used in circuit

<h4 id="Circuit.n_ticks" class="doc_header"><code>Circuit.n_ticks</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Number of ticks

# Circuit helper functions
Sometimes it is useful to get a list of all the qubits which are "touched" in a circuit. From this we can for example infer the total number of qubits in a circuit without ever specifying them directly. Now, we define a `Circuit` as a list of ticks where each tick is represented by a dictionary and the values of these dictionaries contain sets of qubits. These sets can either contain integers or tuples of two integers of qubit indices. Thus, we need a way to unpack (or flatten) all these tuples first before we can merge the indices together in a list. `unpack` accomplishes this recursively.

In [ ]:
#export
def unpack(seq):
    """Generator for all qubits defined in a (sub)circuit"""
    
    if isinstance(seq, (tuple,set,list,Circuit)):
        yield from (x for y in seq for x in unpack(y))
    elif isinstance(seq, dict):
        yield from (x for v in seq.values() for y in v for x in unpack(y))
    else:
        yield seq

Let's see how it works. We give it a list of dictionaries as a representation for some circuit.

In [ ]:
circuit = [{'H': [1, 2, 3]}, {'X': [0, 1], 'CNOT': [(2, 3), (1,5)]}]
print(list(unpack(circuit)))

[1, 2, 3, 0, 1, 2, 3, 1, 5]


And for several circuits, for example as we find them in a protocol.

In [ ]:
c1 = [{'H': [1, 2, 3]}, {'X': [0, 1], 'CNOT': [(2, 3), (1,5)]}]
c2 = [{'Y': [6,1,3]}]
cs = [c1,c2]

set(q for c in cs for q in unpack(c))

{0, 1, 2, 3, 5, 6}

As we can see, the output does, exactly as we expected, contains a list of all the qubits in the order we defined them in the circuit where tuples are correctly unpacked as well.

Another useful method which is of particular interest for the creation of multi-parameter noise models is `partition`. This function allows us to scan a `Circuit` for certain gates and return a list of `(tick_index,qubits)`-tuples which represent a certain `partition` of a circuit. For example we could pass the gate symbols for all two-qubit gates to `partition` to retrieve a list of all circuit locations where two-qubit gates are present. This list can be subsequently used for example to randomly select faulty locations with some probability.

In [ ]:
#export
def partition(circuit,gates):
    """Find all (tick,qubit) circuit locations for list of `gates`"""
    
    return [(t_idx,q) for (t_idx,tick) in enumerate(circuit) 
            for g,qs in tick.items() for q in qs if g in gates]

In [ ]:
print(partition(circuit, ["X"]))
print(partition(circuit, ["X","H"]))
print(partition(circuit, ["CNOT"]))

[(1, 0), (1, 1)]
[(0, 1), (0, 2), (0, 3), (1, 0), (1, 1)]
[(1, (2, 3)), (1, (1, 5))]


If we want to use circuits as keys in dictionaries we first need to create a hash, i.e. a unique ID for a particular circuit. As a circuit is in general a mutable object we cannot hash it itself. The external function `make_hash` generates a hash for a circuit.

In [ ]:
#hide
#export
def freeze(obj):
    if isinstance(obj, dict): # dicts to frozensets
        return frozenset({k:freeze(v) for k,v in obj.items()}.items())
    if isinstance(obj, (set,tuple,list)):
        return tuple([freeze(v) for v in obj]) # lists sets and tuples to tuples
    return obj # hashable object

In [ ]:
#export
def make_hash(circuit):
    return hashlib.sha1(str(freeze(circuit)).encode('UTF-8')).hexdigest()[:10]

In [ ]:
circuit = [{'H': [1, 2, 3]}, {'X': [0, 1], 'CNOT': [(2, 3), (1,5)]}]
circuit2 = [{'H': [1, 2, 3]}, {'X': [0, 1]}]

print(make_hash(circuit))
print(make_hash(circuit2))

459b14736e
4d0c78074f
